# Take-home Assignment

Analyse the current Know Your Customer (KYC) process dataset, identify inefficiencies, and propose actionable solutions to fix the potential issues found. Draft a word document or presentation about your findings and recommendations.

**Context**

As part of the user onboarding process, DolarApp performs KYC (Know Your Customer) to all of its users, with the goal of protecting the platform and the financial system. During this process, DolarApp will ask for an image of ID and selfie to the user which then will be used to do the following checks:
ID verification
Liveness verification
Identity verification
Watchlist Screening

Workflow execution consists of a chain of multiple capability executions (usability, extraction, image checks,...) with some capabilities requiring the results of those executed earlier in the chain.

Because of these dependencies, some capabilities should not be executed if any of the previous ones were not successful because they were REJECTED or NOT_EXECUTED.

If, for example, usability has passed, but extraction got rejected with the reason TECHNICAL_ERROR, the consequent imageChecks and dataChecks cannot be executed because of PRECONDITION_NOT_FULFILLED. The precondition in this case would be to successfully pass extraction.

Current dependencies are:

usability –> extraction -> imageChecks –> dataChecks

usability –> liveness

usability –> similarity

usability –> authentication

usability –> extraction -> imageChecks –> watchlistScreening

usability –> extraction -> imageChecks –> addressValidation

usability –> extraction -> imageChecks –> proofOfResidency

usability –> extraction -> imageChecks –> drivingLicenseVerification

Note: In some cases values may be uploaded as a part of a DATA credential, rather than being extracted from an ID or DOCUMENT. In those situations the dependencies listed above may not apply.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd
summary = pd.read_csv('gdrive/My Drive/KYC_summary.csv')
details = pd.read_csv('gdrive/My Drive/KYC_details.csv')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
summary.head(5)
# 52075 rows by 3 columns

,date_,user_reference,decision_type
0,2023-07-25T00:00:00Z,1.696199e+10,PASSED
1,2023-07-25T00:00:00Z,8.698474e+11,PASSED
2,2023-07-25T00:00:00Z,3.213417e+11,PASSED
3,2023-07-25T00:00:00Z,2.134640e+11,PASSED
4,2023-07-25T00:00:00Z,3.381720e+11,PASSED


In [ ]:
details.head(5)
# 52075 rows by 19 columns

,user_reference,decision_label,usability_decision,usability_decision_details,image_checks_decision,image_checks_decision_details,extraction_decision,extraction_decision_details,data_checks_decision,data_checks_decision_details,liveness_decision,liveness_decision_details,similarity_decision,similarity_decision_details,watchlist_screening_decision,data_type,data_sub_type,data_issuing_country,year_birth
0,1.696199e+10,PASSED,PASSED,OK,PASSED,OK,PASSED,OK,PASSED,OK,PASSED,OK,PASSED,MATCH,PASSED,ID_CARD,ELECTORAL_ID,MEX,1980.0
1,8.698474e+11,PASSED,PASSED,OK,PASSED,OK,PASSED,OK,PASSED,OK,PASSED,OK,PASSED,MATCH,PASSED,DRIVING_LICENSE,REGULAR_DRIVING_LICENSE,MEX,1973.0
2,3.213417e+11,PASSED,PASSED,OK,PASSED,OK,PASSED,OK,PASSED,OK,PASSED,OK,PASSED,MATCH,PASSED,ID_CARD,NATIONAL_ID,MEX,1995.0
3,2.134640e+11,PASSED,PASSED,OK,PASSED,OK,PASSED,OK,PASSED,OK,PASSED,OK,PASSED,MATCH,PASSED,ID_CARD,ELECTORAL_ID,MEX,1984.0
4,3.381720e+11,PASSED,PASSED,OK,PASSED,OK,PASSED,OK,PASSED,OK,PASSED,OK,PASSED,MATCH,PASSED,ID_CARD,NATIONAL_ID,MEX,1962.0


## Exploratory Data Analysis

There are a few columns which have missing values.

1.   watchlist_screening_decision
2.   data_type
3.   data_sub_type
4.   year_birth




In [ ]:
df = pd.merge(details, summary, on='user_reference', how='left')
df.head(5) # 52075 rows by 21 columns


,user_reference,decision_label,usability_decision,usability_decision_details,image_checks_decision,image_checks_decision_details,extraction_decision,extraction_decision_details,data_checks_decision,data_checks_decision_details,...,liveness_decision_details,similarity_decision,similarity_decision_details,watchlist_screening_decision,data_type,data_sub_type,data_issuing_country,year_birth,date_,decision_type
0,1.696199e+10,PASSED,PASSED,OK,PASSED,OK,PASSED,OK,PASSED,OK,...,OK,PASSED,MATCH,PASSED,ID_CARD,ELECTORAL_ID,MEX,1980.0,2023-07-25T00:00:00Z,PASSED
1,8.698474e+11,PASSED,PASSED,OK,PASSED,OK,PASSED,OK,PASSED,OK,...,OK,PASSED,MATCH,PASSED,DRIVING_LICENSE,REGULAR_DRIVING_LICENSE,MEX,1973.0,2023-07-25T00:00:00Z,PASSED
2,3.213417e+11,PASSED,PASSED,OK,PASSED,OK,PASSED,OK,PASSED,OK,...,OK,PASSED,MATCH,PASSED,ID_CARD,NATIONAL_ID,MEX,1995.0,2023-07-25T00:00:00Z,PASSED
3,2.134640e+11,PASSED,PASSED,OK,PASSED,OK,PASSED,OK,PASSED,OK,...,OK,PASSED,MATCH,PASSED,ID_CARD,ELECTORAL_ID,MEX,1984.0,2023-07-25T00:00:00Z,PASSED
4,3.381720e+11,PASSED,PASSED,OK,PASSED,OK,PASSED,OK,PASSED,OK,...,OK,PASSED,MATCH,PASSED,ID_CARD,NATIONAL_ID,MEX,1962.0,2023-07-25T00:00:00Z,PASSED


In [ ]:
df['data_issuing_country'].value_counts()

,count
data_issuing_country,
MEX,29706
ARG,22369


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52075 entries, 0 to 52074
Data columns (total 21 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   user_reference                 52075 non-null  float64
 1   decision_label                 52075 non-null  object 
 2   usability_decision             52075 non-null  object 
 3   usability_decision_details     52075 non-null  object 
 4   image_checks_decision          52075 non-null  object 
 5   image_checks_decision_details  52075 non-null  object 
 6   extraction_decision            52075 non-null  object 
 7   extraction_decision_details    52075 non-null  object 
 8   data_checks_decision           52075 non-null  object 
 9   data_checks_decision_details   52075 non-null  object 
 10  liveness_decision              52075 non-null  object 
 11  liveness_decision_details      52075 non-null  object 
 12  similarity_decision            52075 non-null 

In [ ]:
capability_columns = ['usability_decision', 'image_checks_decision','extraction_decision','data_checks_decision','liveness_decision','similarity_decision','watchlist_screening_decision']

summary_data = []

for col in capability_columns:
  counts = df[col].value_counts()
  total = len(df)

  summary_data.append({
      'capability': col,
      'total': total,
      'PASSED': counts.get('PASSED', 0),
      'REJECTED': counts.get('REJECTED', 0),
      'NOT_EXECUTED': counts.get('NOT_EXECUTED', 0),
      'WARNING': counts.get('WARNING',0),
      'Pass Rate': round((counts.get('PASSED',0) / total)*100,2),
      'Rejection Rate': round((counts.get('REJECTED',0) / total)*100,2),
      'Not Executed Rate': round((counts.get('NOT_EXECUTED',0) / total)*100,2)
  })

summary_df = pd.DataFrame(summary_data)

print(summary_df)

                     capability  total  PASSED  REJECTED  NOT_EXECUTED  \
0            usability_decision  52075   49647       933           708   
1         image_checks_decision  52075   47219       945          3910   
2           extraction_decision  52075   48164         0          3910   
3          data_checks_decision  52075   47210         9          4855   
4             liveness_decision  52075   50894      1032             1   
5           similarity_decision  52075   47926      1104          2637   
6  watchlist_screening_decision  52075   45399         0             1   

   WARNING  Pass Rate  Rejection Rate  Not Executed Rate  
0      787      95.34            1.79               1.36  
1        0      90.67            1.81               7.51  
2        0      92.49            0.00               7.51  
3        0      90.66            0.02               9.32  
4      148      97.73            1.98               0.00  
5      408      92.03            2.12               5

In [ ]:
success_rate = (df['decision_type']  == 'PASSED').mean() *100
print(f"Success Rate: {success_rate:.2f}%")

Success Rate: 86.08%


With an overall success rate of 86%, there is a 13% rejection rate with the remaining 1% being not executed. 13% rejection rate is quite high.



# Capability Analysis

## Root Cause Analysis

Usability is the very first dependanct for all workflows. There are 933 rejected and 708 not executed. Why is this? Note, there are also 787 warning rows.

In [ ]:
filtered_df = df[df['usability_decision'].isin(["REJECTED","NOT_EXECUTED","WARNING"])]
filtered_df.head(10)


,user_reference,decision_label,usability_decision,usability_decision_details,image_checks_decision,image_checks_decision_details,extraction_decision,extraction_decision_details,data_checks_decision,data_checks_decision_details,...,liveness_decision_details,similarity_decision,similarity_decision_details,watchlist_screening_decision,data_type,data_sub_type,data_issuing_country,year_birth,date_,decision_type
28,4.260775e+11,REJECTED,REJECTED,MISSING_MANDATORY_DATAPOINTS,NOT_EXECUTED,PRECONDITION_NOT_FULFILLED,NOT_EXECUTED,PRECONDITION_NOT_FULFILLED,NOT_EXECUTED,PRECONDITION_NOT_FULFILLED,...,OK,NOT_EXECUTED,PRECONDITION_NOT_FULFILLED,NaN,NaN,NaN,MEX,NaN,2023-07-25T00:00:00Z,REJECTED
52,1.053373e+11,REJECTED,REJECTED,MISSING_MANDATORY_DATAPOINTS,NOT_EXECUTED,PRECONDITION_NOT_FULFILLED,NOT_EXECUTED,PRECONDITION_NOT_FULFILLED,NOT_EXECUTED,PRECONDITION_NOT_FULFILLED,...,OK,NOT_EXECUTED,PRECONDITION_NOT_FULFILLED,NaN,NaN,NaN,MEX,NaN,2023-07-25T00:00:00Z,REJECTED
149,8.655730e+11,REJECTED,REJECTED,GLARE,NOT_EXECUTED,PRECONDITION_NOT_FULFILLED,NOT_EXECUTED,PRECONDITION_NOT_FULFILLED,NOT_EXECUTED,PRECONDITION_NOT_FULFILLED,...,OK,NOT_EXECUTED,PRECONDITION_NOT_FULFILLED,NaN,NaN,NaN,MEX,NaN,2023-07-25T00:00:00Z,REJECTED
158,5.121048e+10,REJECTED,REJECTED,GLARE,NOT_EXECUTED,PRECONDITION_NOT_FULFILLED,NOT_EXECUTED,PRECONDITION_NOT_FULFILLED,NOT_EXECUTED,PRECONDITION_NOT_FULFILLED,...,OK,NOT_EXECUTED,PRECONDITION_NOT_FULFILLED,NaN,NaN,NaN,MEX,NaN,2023-07-25T00:00:00Z,REJECTED
163,4.156861e+11,REJECTED,WARNING,UNSUPPORTED_DOCUMENT_TYPE,NOT_EXECUTED,PRECONDITION_NOT_FULFILLED,NOT_EXECUTED,PRECONDITION_NOT_FULFILLED,NOT_EXECUTED,PRECONDITION_NOT_FULFILLED,...,OK,PASSED,MATCH,NaN,NaN,NaN,MEX,NaN,2023-07-25T00:00:00Z,REJECTED
293,1.592213e+11,REJECTED,REJECTED,MISSING_MANDATORY_DATAPOINTS,NOT_EXECUTED,PRECONDITION_NOT_FULFILLED,NOT_EXECUTED,PRECONDITION_NOT_FULFILLED,NOT_EXECUTED,PRECONDITION_NOT_FULFILLED,...,OK,NOT_EXECUTED,PRECONDITION_NOT_FULFILLED,NaN,NaN,NaN,MEX,NaN,2023-07-25T00:00:00Z,REJECTED
391,2.686824e+11,REJECTED,WARNING,UNSUPPORTED_DOCUMENT_TYPE,NOT_EXECUTED,PRECONDITION_NOT_FULFILLED,NOT_EXECUTED,PRECONDITION_NOT_FULFILLED,NOT_EXECUTED,PRECONDITION_NOT_FULFILLED,...,OK,PASSED,MATCH,NaN,NaN,NaN,MEX,NaN,2023-07-25T00:00:00Z,REJECTED
461,7.597772e+11,REJECTED,REJECTED,MISSING_MANDATORY_DATAPOINTS,NOT_EXECUTED,PRECONDITION_NOT_FULFILLED,NOT_EXECUTED,PRECONDITION_NOT_FULFILLED,NOT_EXECUTED,PRECONDITION_NOT_FULFILLED,...,OK,NOT_EXECUTED,PRECONDITION_NOT_FULFILLED,NaN,NaN,NaN,MEX,NaN,2023-07-25T00:00:00Z,REJECTED
485,4.354358e+11,REJECTED,WARNING,UNSUPPORTED_DOCUMENT_TYPE,NOT_EXECUTED,PRECONDITION_NOT_FULFILLED,NOT_EXECUTED,PRECONDITION_NOT_FULFILLED,NOT_EXECUTED,PRECONDITION_NOT_FULFILLED,...,OK,PASSED,MATCH,NaN,NaN,NaN,MEX,NaN,2023-07-25T00:00:00Z,REJECTED
512,7.610897e+11,REJECTED,REJECTED,MISSING_MANDATORY_DATAPOINTS,NOT_EXECUTED,PRECONDITION_NOT_FULFILLED,NOT_EXECUTED,PRECONDITION_NOT_FULFILLED,NOT_EXECUTED,PRECONDITION_NOT_FULFILLED,...,OK,NOT_EXECUTED,PRECONDITION_NOT_FULFILLED,NaN,NaN,NaN,MEX,NaN,2023-07-25T00:00:00Z,REJECTED


Because the usability has not passed, the workflow downstream will not operate due to precondition not fulfilled.

Take a look to see why they were rejected, not executed or warning.

In [ ]:
usability_table = pd.crosstab(df['usability_decision_details'],df['usability_decision'])
usability_table

usability_decision,NOT_EXECUTED,PASSED,REJECTED,WARNING
usability_decision_details,,,,
BLURRED,0,0,215,0
DAMAGED_DOCUMENT,0,0,50,0
GLARE,0,0,96,0
MISSING_MANDATORY_DATAPOINTS,0,0,407,0
MISSING_PAGE,0,0,138,0
NOT_A_DOCUMENT,0,0,3,0
NOT_UPLOADED,301,0,0,0
OK,0,49646,0,0
PART_OF_DOCUMENT_HIDDEN,0,0,2,0


In [ ]:
df['usability_decision'].value_counts()

,count
usability_decision,
PASSED,49647
REJECTED,933
WARNING,787
NOT_EXECUTED,708


From the usability capability, many were rejected for blurred images, damaged documents, glare, missing mandatory datapoints, missing pages.

407 were not executed due to technical error. This is 0.8% of all users.

Furthermore, what seems to be a problem for usability is unsupported document types (509 people), and undetermined liveness (278).

In [ ]:
liveness_table = pd.crosstab(df['liveness_decision_details'],df['liveness_decision'])
liveness_table

liveness_decision,NOT_EXECUTED,PASSED,REJECTED,WARNING
liveness_decision_details,,,,
BAD_QUALITY,0,0,0,148
FACE_NOT_FULLY_VISIBLE,0,0,53,0
ID_USED_AS_SELFIE,0,0,14,0
OK,0,50894,0,0
TECHNICAL_ERROR,1,0,0,0
liveness_UNDETERMINED,0,0,965,0


In [ ]:
df['liveness_decision_details'].value_counts()

,count
liveness_decision_details,
OK,50894
liveness_UNDETERMINED,965
BAD_QUALITY,148
FACE_NOT_FULLY_VISIBLE,53
ID_USED_AS_SELFIE,14
TECHNICAL_ERROR,1


For workflow 2 (usability -> liveness):

Only 1 not executed due to technical error. For those that were rejected; 53 were face not fully visible, 14 were from ID being used as selfie and 965 were from liveness undetermined.


There is a 1.85% of undetermined liveness out of every user. Out of the people that were rejected this is 93.5%.This is a major issue.

In [ ]:
extraction_table = pd.crosstab(df['extraction_decision_details'],df['extraction_decision'])
extraction_table

extraction_decision,NOT_EXECUTED,PASSED,PASSES
extraction_decision_details,,,
OK,0,48164,1
PRECONDITION_NOT_FULFILLED,3909,0,0
TECHNICAL_ERROR,1,0,0


In [ ]:
image_checks_table = pd.crosstab(df['image_checks_decision_details'],df['image_checks_decision'])
image_checks_table

image_checks_decision,NOT_EXECUTED,PASSED,PASSES,REJECTED
image_checks_decision_details,,,,
DIGITAL_COPY,0,0,0,225
FAKE,0,0,0,3
MANIPULATED_DOCUMENT,0,0,0,537
MANIPULATED_DOCUMENT_DOB,0,0,0,57
MANIPULATED_DOCUMENT_DOCUMENT_NUMBER,0,0,0,19
MANIPULATED_DOCUMENT_EXPIRY,0,0,0,2
MANIPULATED_DOCUMENT_NAME,0,0,0,6
MANIPULATED_DOCUMENT_PHOTO,0,0,0,61
MANIPULATED_DOCUMENT_SECURITY_CHECKS,0,0,0,13


There is an issue due to manipulation of documents.

In [ ]:
similarity_table = pd.crosstab(df['similarity_decision_details'],df['similarity_decision'])
similarity_table

similarity_decision,NOT_EXECUTED,PASSED,REJECTED,WARNING
similarity_decision_details,,,,
MATCH,0,47917,0,0
NOT_POSSIBLE,0,0,0,408
NO_MATCH,0,0,1104,0
OK,0,9,0,0
PRECONDITION_NOT_FULFILLED,2636,0,0,0
TECHNICAL_ERROR,1,0,0,0


1104 rejected due to no match, this 2.3%. Identity matching is an issue.

In [ ]:
data_checks_table = pd.crosstab(df['data_checks_decision_details'],df['data_checks_decision'])
data_checks_table

data_checks_decision,NOT_EXECUTED,PASSED,PASSES,REJECTED
data_checks_decision_details,,,,
MISMATCHING_DATAPOINTS,0,0,0,6
MISMATCH_HRZ_MRZ_DATA,0,0,0,3
OK,0,47210,1,0
PRECONDITION_NOT_FULFILLED,4854,0,0,0
TECHNICAL_ERROR,1,0,0,0


In [ ]:
df['watchlist_screening_decision'].value_counts()

,count
watchlist_screening_decision,
PASSED,45399
WARNING,69
NOT_EXECUTED,1


# Analysis

The biggest issues are identity matching, manipulation of documents, and undetermined liveness. Slight percentage of technical error being an issue <1.0%.


How does the onboarding process affect DolarApp?

Revenue = Customers Onboarded * Average transaction Volume * Take Rate

In [ ]:
def overall_performance(df):

  total_users = len(df)
  decision_overall = df['decision_type'].value_counts().sort_values(ascending=False)

  for decision, count in decision_overall.items():
    percentage = (count / total_users) * 100
    print(f"{decision}: {count:,} ({percentage:.2f}%)")

  successful = (decision_overall.get('PASSED', 0) + decision_overall.get('APPROVED', 0))
  rejected = decision_overall.get('REJECTED', 0)
  warning = decision_overall.get('WARNING', 0)

  conversion_rate = (successful / total_users) * 100
  rejection_rate = (rejected / total_users) * 100
  warning_rate = (warning / total_users) * 100

  print(f"Overall Approval Rate: {conversion_rate:.2f}%")
  print(f"Rejection Rate: {rejection_rate:.2f}%")
  print(f"Warning Rate: {warning_rate:.2f}%")

In [ ]:
overall_performance(df)

PASSED: 44,825 (86.08%)
REJECTED: 6,772 (13.00%)
APPROVED: 4 (0.01%)
Overall Approval Rate: 86.09%
Rejection Rate: 13.00%
Warning Rate: 0.91%


In [ ]:
def capability_analysis(df):

        check_types = [
            {'name': 'Usability Check', 'field': 'usability_decision', 'capability': 'usability'},
            {'name': 'Image Quality Check', 'field': 'image_checks_decision', 'capability': 'image_checks'},
            {'name': 'Data Extraction Check', 'field': 'extraction_decision', 'capability': 'extraction'},
            {'name': 'Data Validation Check', 'field': 'data_checks_decision', 'capability': 'data_checks'},
            {'name': 'Liveness Detection', 'field': 'liveness_decision', 'capability': 'liveness'},
            {'name': 'Similarity Check', 'field': 'similarity_decision', 'capability': 'similarity'},
            {'name': 'Watchlist Screening', 'field': 'watchlist_screening_decision', 'capability': 'watchlist_screening'}
        ]

        bottleneck_analysis = []
        total_records = len(df)


        for check in check_types:
            field = check['field']
            check_decisions = df[field].value_counts()

            print(f"{check['name']}:")
            for decision, count in check_decisions.items():
                percentage = (count / total_records) * 100
                print(f"   {decision}: {count:,} ({percentage:.2f}%)")

            explicit_failures = (
                check_decisions.get('FAILED', 0) +
                check_decisions.get('REJECTED', 0)
            )
            failure_rate = (explicit_failures / total_records) * 100

        return

In [ ]:
capability_analysis(df)

Usability Check:
   PASSED: 49,647 (95.34%)
   REJECTED: 933 (1.79%)
   NOT_EXECUTED: 708 (1.36%)
Image Quality Check:
   PASSED: 47,219 (90.67%)
   NOT_EXECUTED: 3,910 (7.51%)
   REJECTED: 945 (1.81%)
   PASSES: 1 (0.00%)
Data Extraction Check:
   PASSED: 48,164 (92.49%)
   NOT_EXECUTED: 3,910 (7.51%)
   PASSES: 1 (0.00%)
Data Validation Check:
   PASSED: 47,210 (90.66%)
   NOT_EXECUTED: 4,855 (9.32%)
   REJECTED: 9 (0.02%)
   PASSES: 1 (0.00%)
Liveness Detection:
   PASSED: 50,894 (97.73%)
   REJECTED: 1,032 (1.98%)
   NOT_EXECUTED: 1 (0.00%)
Similarity Check:
   PASSED: 47,926 (92.03%)
   NOT_EXECUTED: 2,637 (5.06%)
   REJECTED: 1,104 (2.12%)
Watchlist Screening:
   PASSED: 45,399 (87.18%)
   NOT_EXECUTED: 1 (0.00%)


In [ ]:
def country(df):
        country_analysis = []
        countries = df['data_issuing_country'].unique()
        countries = [c for c in countries if pd.notna(c)]
        total_records = len(df)

        for country in countries:
            country_data = df[df['data_issuing_country'] == country]

            total = len(country_data)
            passed = len(country_data[country_data['decision_label'] == 'PASSED'])
            rejected = len(country_data[country_data['decision_label'] == 'REJECTED'])
            warning = len(country_data[country_data['decision_label'] == 'WARNING'])

            pass_rate = (passed / total) * 100
            rejection_rate = (rejected / total) * 100
            warning_rate = (warning / total) * 100
            market_share = (total / total_records) * 100

            country_analysis.append({
                'country': country,
                'total_applications': total,
                'passed_applications': passed,
                'rejected_applications': rejected,
                'warning_applications': warning,
                'pass_rate': pass_rate,
                'rejection_rate': rejection_rate,
                'warning_rate': warning_rate,
                'market_share': market_share,
            })

            print(f"   {country}:")
            print(f"   Applications: {total:,}")
            print(f"   Pass Rate: {pass_rate:.2f}%")
            print(f"   Rejection Rate: {rejection_rate:.2f}%")
            print(f"   Warning Rate: {warning_rate:.2f}%")

        country_analysis.sort(key=lambda x: x['total_applications'], reverse=True)

        return

In [ ]:
country(df)

   MEX:
   Applications: 29,706
   Pass Rate: 81.54%
   Rejection Rate: 17.36%
   Warning Rate: 1.07%
   ARG:
   Applications: 22,369
   Pass Rate: 92.07%
   Rejection Rate: 7.22%
   Warning Rate: 0.70%


In [ ]:
def data_type(df):
        # Taking a look at the type of documents submitted
        # Filter out null document types
        df_without_nulls = df.dropna(subset=['data_type'])

        doc_type_analysis = []
        doc_types = df_without_nulls['data_type'].unique()


        for doc_type in doc_types:
            doc_data = df_without_nulls[df_without_nulls['data_type'] == doc_type]

            total = len(doc_data)
            passed = len(doc_data[doc_data['decision_label'] == 'PASSED'])
            rejected = len(doc_data[doc_data['decision_label'] == 'REJECTED'])

            pass_rate = (passed / total) * 100
            rejection_rate = (rejected / total) * 100

            subtypes = doc_data['data_sub_type'].value_counts()

            doc_type_analysis.append({
                'document_type': doc_type,
                'total_applications': total,
                'pass_rate': pass_rate,
                'rejection_rate': rejection_rate,
                'subtypes': subtypes.to_dict()
            })
            print(f"{doc_type}:")
            print(f"Applications: {total:,}")
            print(f"Pass Rate: {pass_rate:.2f}%")
            print(f"Rejection Rate: {rejection_rate:.2f}%")
        return

In [ ]:
data_type(df)

ID_CARD:
Applications: 44,638
Pass Rate: 92.62%
Rejection Rate: 6.35%
DRIVING_LICENSE:
Applications: 972
Pass Rate: 89.51%
Rejection Rate: 9.36%
PASSPORT:
Applications: 2,660
Pass Rate: 96.43%
Rejection Rate: 3.23%
VISA:
Applications: 24
Pass Rate: 95.83%
Rejection Rate: 4.17%


We can ignore VISA since there were only 24 applications, too small of a sample to take any learnings from.

In [ ]:
driving_license_rejection_analysis = pd.crosstab(
    df[df['data_type'] == 'DRIVING_LICENSE']['decision_label'],
    df[df['data_type'] == 'DRIVING_LICENSE']['usability_decision_details']
)

driving_license_rejection_analysis

usability_decision_details,NOT_UPLOADED,OK,TECHNICAL_ERROR,liveness_UNDETERMINED
decision_label,,,,
PASSED,6,864,0,0
REJECTED,2,74,7,8
WARNING,0,11,0,0


In [ ]:
df['data_type'].value_counts()

,count
data_type,
ID_CARD,44638
PASSPORT,2660
DRIVING_LICENSE,972
VISA,24


# Financial Impact

In [ ]:
# Financial Impact analysis

avg_transaction_size = 500  #USD
avg_transactions = 50 # avg transactions per user
total_volume_per_customer = avg_transaction_size * avg_transactions # 10 transactions avg per user

fx_spread = 0.0025
fx_revenue_per_customer = total_volume_per_customer * fx_spread

fee_revenue_per_customer = 3 * avg_transactions

total_revenue_per_customer = fee_revenue_per_customer + fx_revenue_per_customer
print(total_revenue_per_customer)

212.5
